In [227]:
# ! git clone https://github.com/Angeluz-07/audio-processing-files.git
# ! ls

In [228]:
# !wget "https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz?"
# !tar xvf "/content/genres.tar.gz?" -C "/content/"

In [229]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [230]:
# from glob import glob

# #directories of normal audios
# nowvar = "rock"
# norm_data_dir = '/content/genres/'+nowvar+'/'
# norm_audio_files = glob(norm_data_dir + '*.au')

In [231]:
# import librosa
# from librosa import feature
# import numpy as np

# fn_list_i = [
#              feature.chroma_stft, feature.spectral_centroid,
#              feature.spectral_bandwidth, feature.spectral_rolloff
#             ]

# fn_list_ii = [librosa.feature.rms, feature.zero_crossing_rate]

# def get_feature_vector(y,sr): 
#    feat_vect_i = [ np.mean(funct(y,sr)) for funct in fn_list_i]
#    feat_vect_ii = [np.mean(funct(y)) for funct in fn_list_ii] 
#    feature_vector = feat_vect_i + feat_vect_ii 
#    return feature_vector

In [232]:
# norm_audios_feat = []
# for file in norm_audio_files:
#    y , sr = librosa.load(file, sr=None)
#    feature_vector = get_feature_vector(y, sr)
#    norm_audios_feat.append(feature_vector) 

In [233]:
# import csv
# norm_output = nowvar+'.csv'
# header =[
#  'chroma_stft',
#  'spectral_centroid',
#  'spectral_bandwidth',
#  'spectral_rolloff',
#  'rmse',
#  'zero_crossing_rate'
# ]
# with open(norm_output,'+w') as f:
#  csv_writer = csv.writer(f, delimiter = ',')
#  csv_writer.writerow(header)
#  csv_writer.writerows(norm_audios_feat)

In [234]:
blues = pd.read_csv("/content/blues.csv")
classical = pd.read_csv("/content/classical.csv")
country = pd.read_csv("/content/country.csv")
disco = pd.read_csv("/content/disco.csv")
hiphop = pd.read_csv("/content/hiphop.csv")
jazz = pd.read_csv("/content/jazz.csv")
metal = pd.read_csv("/content/metal.csv")
pop = pd.read_csv("/content/pop.csv")
reggae = pd.read_csv("/content/reggae.csv")
rock = pd.read_csv("/content/rock.csv")

In [235]:
final_df = pd.DataFrame(columns = rock.columns)

In [236]:
genres = [blues, 
          classical, 
          country, 
          disco, 
          hiphop, 
          jazz,
          metal, 
          pop,
          reggae,
          rock]

final_df = pd.concat(genres)

In [237]:
final_df.sample(20)

,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,rmse,zero_crossing_rate
9,0.517750,2967.125658,2556.786139,5804.380937,0.089974,0.121885
30,0.314550,1397.821606,1896.363950,3100.931133,0.107259,0.048583
63,0.479806,3084.761864,2970.591186,6778.925675,0.213130,0.136444
62,0.418531,1995.164243,2295.668202,4557.786387,0.078408,0.070276
70,0.302543,2032.381163,1850.490121,3838.955441,0.039069,0.101047
35,0.408241,2590.730306,2078.710859,4889.402711,0.089583,0.161723
57,0.516547,3338.581900,2697.128636,6670.863091,0.267380,0.186739
62,0.358566,1981.662203,2144.724075,4261.342626,0.053910,0.093536
56,0.506826,3079.127911,3220.914517,7003.520270,0.108968,0.107077
67,0.504624,3501.713729,3027.178552,7199.642286,0.193659,0.186920


In [238]:
convertor = LabelEncoder()
class_list = final_df.iloc[:,-1]
y = convertor.fit_transform(class_list)

In [239]:
fit = StandardScaler()
X = fit.fit_transform(np.array(final_df.iloc[:,1:-1], dtype = float))

In [240]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [241]:
def trainModel (model, epochs, optimizer):
  batch_size = 128

  model.compile(optimizer,
                loss='sparse_categorical_crossentropy',
                metrics='accuracy')
  return model.fit(x_train, y_train,
                 validation_data=(x_test, y_test),
                 epochs=epochs, batch_size=batch_size, verbose=1)

In [242]:
def plot_validate(history):
  print("Validation Accuracy", max(history.history["val accuracy"]))
  pd.DataFrame(history.history).plot(figsize=(12, 6))
  plt.show()

In [243]:
model = keras.models.Sequential([
                             keras.layers.Dense(512, activation='relu',
                                                input_shape=(x_train.shape[1],)),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(256, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(128, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(64, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(10, activation='softmax')
                          ])

print(model.summary())
model_history = trainModel(model=model, epochs=600, optimizer='adam')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               2560      
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8

InvalidArgumentError: ignored

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=128)
print ("The test loss is: ", test_loss)
print("\nThe best test accuracy is :", test_acc*100)